In [23]:
import numpy as np

In [20]:
p = 13
K = GF(p)
n = 5
R = PolynomialRing(K, x, n) #defines n variables over K
R.inject_variables(verbose=False) # makes all the variables ready for use

In [44]:
# generate two m-tuples of inhomogenous multivariate polynomials of degree 2
m = 4

F = []
for i in range(m):
    F.append(R.random_element(degree=2))
F

[-6*x2^2 + 4*x0*x3 - 4*x4^2,
 -x1*x2 - 4*x2^2 + 6*x1*x4 + 6*x0,
 -5*x2*x3 - 4*x3^2 - 3*x0 - 5*x2 + 4,
 4*x0*x2 + x3*x4 - 2*x3 + 5*x4 - 5]

In [46]:
# G = GL(m, K)
T = GL(m, K).random_element()
S = GL(n, K).random_element()
print(T,"\n",S)

print(T*F[1])

[ 6  6  0  9]
[ 3  2  5 11]
[ 2  8  4  2]
[ 1  6  6 12] 
 [ 3 11 11  3  5]
[ 4  5  4  9  0]
[ 2  0  0  9  9]
[11 11  0 12  7]
[ 9  3  8  0  5]
[-6*x1*x2 + 2*x2^2 - 3*x1*x4 - 3*x0 -6*x1*x2 + 2*x2^2 - 3*x1*x4 - 3*x0                                  0  4*x1*x2 + 3*x2^2 + 2*x1*x4 + 2*x0]
[  -3*x1*x2 + x2^2 + 5*x1*x4 + 5*x0     -2*x1*x2 + 5*x2^2 - x1*x4 - x0 -5*x1*x2 + 6*x2^2 + 4*x1*x4 + 4*x0      2*x1*x2 - 5*x2^2 + x1*x4 + x0]
[    -2*x1*x2 + 5*x2^2 - x1*x4 - x0  5*x1*x2 - 6*x2^2 - 4*x1*x4 - 4*x0 -4*x1*x2 - 3*x2^2 - 2*x1*x4 - 2*x0     -2*x1*x2 + 5*x2^2 - x1*x4 - x0]
[  -x1*x2 - 4*x2^2 + 6*x1*x4 + 6*x0 -6*x1*x2 + 2*x2^2 - 3*x1*x4 - 3*x0 -6*x1*x2 + 2*x2^2 - 3*x1*x4 - 3*x0    x1*x2 + 4*x2^2 - 6*x1*x4 - 6*x0]
